# SQL Case Study

In [5]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [6]:
# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Save the table names to a list: table_names
inspector = inspect(engine)
table_names = inspector.get_table_names()

# Print the table names to the shell
print(table_names)

['Bookings', 'Facilities', 'Members']


In [8]:
# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Facilities")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

# Print the head of the DataFrame df
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [9]:
# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM Members", engine)

# Print head of DataFrame
print(df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


# Questions

### Q10: Produce a list of facilities with a total revenue less than 1000.<br>The output of facility name and total revenue, sorted by revenue. <br>Remember that there's a different cost for guests and members!

In [11]:
Q10_sql = "SELECT f.name, "\
            "SUM(CASE WHEN b.memid = 0 "\
                    "THEN f.guestcost * b.slots "\
                "ELSE f.membercost * b.slots END) AS total_revenue "\
        "FROM Bookings AS b " \
        "INNER JOIN Facilities AS f "\
        "ON b.facid = f.facid "\
        "GROUP BY f.name "\
        "HAVING total_revenue < 1000 "\
        "ORDER BY total_revenue;"
Q10_df = pd.read_sql_query(Q10_sql, engine)
Q10_df

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [15]:
Q11_sql = "SELECT (m.surname || ', ' || m.firstname) AS member_name, "\
                "CASE WHEN m.recommendedby = 0  "\
                    "THEN 'None' "\
                "ELSE (r.surname || ', ' || r.firstname) END AS recommended_by "\
            "FROM Members AS m "\
            "LEFT JOIN Members AS r "\
                "ON r.memid = m.recommendedby "\
            "WHERE m.surname <> 'GUEST' "\
            "ORDER BY recommended_by;"
Q11_df = pd.read_sql_query(Q11_sql, engine)
Q11_df

,member_name,recommended_by
0,"Smith, Darren",None
1,"Smith, Tracy",None
2,"Rownam, Tim",None
3,"Tracy, Burton",None
4,"Farrell, Jemima",None
5,"Farrell, David",None
6,"Tupperware, Hyacinth",None
7,"Smith, Darren",None
8,"Sarwin, Ramnaresh","Bader, Florence"
9,"Coplin, Joan","Baker, Timothy"


### Q12: Find the facilities with their usage by member, but not guests

In [18]:
# Total Facility Usage by Members - Grouped by Facility
Q12a_sql = "SELECT DISTINCT f.name AS facility_name, "\
                "COUNT(*) AS usage_count "\
            "FROM Bookings AS b "\
            "INNER JOIN Members AS m  "\
                "ON b.memid = m.memid "\
            "INNER JOIN Facilities AS f "\
                "ON b.facid = f.facid "\
            "WHERE m.surname <> 'GUEST' "\
            "GROUP BY facility_name "\
            "ORDER BY facility_name;"
Q12a_df = pd.read_sql_query(Q12a_sql, engine)
Q12a_df

,facility_name,usage_count
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [17]:
# Total Facility Usage by Members - Grouped by Facility and Member
Q12b_sql = "SELECT DISTINCT f.name AS facility_name, "\
                "(surname || ', ' || firstname) AS member_name, "\
                "COUNT(*) AS usage_count "\
            "FROM Bookings AS b "\
            "INNER JOIN Members AS m  "\
                "ON b.memid = m.memid "\
            "INNER JOIN Facilities AS f "\
                "ON b.facid = f.facid "\
            "WHERE m.surname <> 'GUEST' "\
            "GROUP BY facility_name, member_name "\
            "ORDER BY facility_name, member_name;"
Q12b_df = pd.read_sql_query(Q12b_sql, engine)
Q12b_df

,facility_name,member_name,usage_count
0,Badminton Court,"Bader, Florence",9
1,Badminton Court,"Baker, Anne",10
2,Badminton Court,"Baker, Timothy",7
3,Badminton Court,"Boothe, Tim",12
4,Badminton Court,"Butters, Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith, Darren",19
198,Tennis Court 2,"Smith, Jack",1
199,Tennis Court 2,"Smith, Tracy",2
200,Tennis Court 2,"Stibbons, Ponder",31


### Q13: Find the facilities usage by month, but not guests

In [21]:
Q13_sql = "SELECT DISTINCT f.name AS facility_name, "\
            "CASE WHEN SUBSTR(b.starttime, 6, 2) = '01' "\
                "THEN 'January' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '02' "\
                "THEN 'February' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '03' "\
                "THEN 'March' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '04' "\
                "THEN 'April' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '05' "\
                "THEN 'May' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '06' "\
                "THEN 'June' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '07' "\
                "THEN 'July' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '08' "\
                "THEN 'August' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '09' "\
                "THEN 'September' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '10' "\
                "THEN 'October' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '11' "\
                "THEN 'November' "\
            "WHEN SUBSTR(b.starttime, 6, 2) = '12' "\
                "THEN 'December' "\
            "ELSE 'Unknown' END AS month_name, "\
            "COUNT(*) AS usage_count "\
        "FROM Bookings AS b "\
        "INNER JOIN Members AS m  "\
        "ON b.memid = m.memid "\
        "INNER JOIN Facilities AS f "\
        "ON b.facid = f.facid "\
        "WHERE m.surname <> 'GUEST' "\
        "GROUP BY facility_name, month_name "\
        "ORDER BY facility_name, SUBSTR(b.starttime, 6, 2);"
Q13_df = pd.read_sql_query(Q13_sql, engine)
Q13_df

,facility_name,month_name,usage_count
0,Badminton Court,July,51
1,Badminton Court,August,132
2,Badminton Court,September,161
3,Massage Room 1,July,77
4,Massage Room 1,August,153
5,Massage Room 1,September,191
6,Massage Room 2,July,4
7,Massage Room 2,August,9
8,Massage Room 2,September,14
9,Pool Table,July,103


In [26]:
test_sql = "SELECT strftime('%m', DATE(starttime)) FROM bookings;"
test_df = pd.read_sql_query(test_sql, engine)
test_df

,"strftime('%m', DATE(starttime))"
0,07
1,07
2,07
3,07
4,07
...,...
4038,09
4039,09
4040,09
4041,09
